# Crucible TEM data viewer

This notebook provides an interactive widget to look through many types of TEM data in a Crucible Google Drive folder. It is designed to work with Google Colab.

Note: Only images are currently supported. Spectra, 3D, and 4D data sets are not supported.

author: Peter Ercius, percius@lbl.gov

In [ ]:
!pip install -q ipympl ncempy==1.11.3

In [ ]:
%matplotlib widget

In [ ]:
# Allow access to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Allow interactive plots
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Necessary imports
# %matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive

import ncempy

# Supported data types (using ncempy.read())
types = ('.emd', '.ser', '.dm3', '.dm4')

## User input required
 - Set `dir_path` to the base path a Shared Drive folder
   - example: /content/drive/Shareddrives/MFP012345/Datasets

In [ ]:
dir_path = Path('/content/drive/Shareddrives/MFP09699/Datasets') # change!

all_files = []
for data_type in types:
    type_files = dir_path.glob('**/*' + data_type)
    all_files.extend(list(type_files))

scanid_tree = {}
for file in all_files:
    scanid_tree[file.stem] = file

print(f'Found {len(scanid_tree)} files')

## Interactive data browser
 - All datasets in the Shared Drive folder are listed in teh dropdown by file name
 - Please be patient. Google Colab interactivity can be slow

In [ ]:
global imax1, ax1

data = ncempy.read(scanid_tree[next(iter(scanid_tree))])

fg1, ax1 = plt.subplots(1,1,figsize=(8,8))
ndim = data['data'].ndim
imax1 = ax1.imshow(np.random.rand(10,10)) # Set the initial image and intensity scaling
ax1.set(xlabel='X ({})'.format(data['pixelUnit'][0]),
        ylabel='Y ({})'.format(data['pixelUnit'][1]));

# Updates the plot
def axUpdate(scanid):
    global imax1, ax1
    print(f'Loading {scanid_tree[str(scanid)].name}')
    data = ncempy.read(scanid_tree[str(scanid)])
    print(f'Data shape: {data["data"].shape}')
    ax1.cla() # need to clear each time due to heterogeneous data

    # Handle 3D data. Only the first image is shown
    if data['data'].ndim == 2:
        im = data['data']
        ix = 0
        iy = 1
    elif data['data'].ndim == 3:
        im = data['data'][0,:,:]
        ix = 1
        iy = 2
    else:
        print('File {}: data not supported'.format(scanid))
        return

    imax1 = ax1.imshow(im)
    fovX = data['pixelSize'][ix]*data['data'].shape[ix]
    fovY = data['pixelSize'][iy]*data['data'].shape[iy]
    imax1.set_extent((0, fovX, fovY, 0))
    ax1.set(xlabel='X ({})'.format(data['pixelUnit'][ix]),
            ylabel='Y ({})'.format(data['pixelUnit'][iy]),
            title=f'{scanid_tree[str(scanid)].name}')
    fg1.canvas.draw()

# Create the slider to update the plot
w = interactive(axUpdate, scanid=scanid_tree.keys())
out = widgets.Output(layout={'border': '1px solid black'})
display(w)